In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Embedding, LSTM, Concatenate, Dropout, TimeDistributed,
                                     Input, Dense, Bidirectional, Layer)
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import Progbar
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
from plotly.offline import init_notebook_mode
import string

2024-03-20 15:37:46.392416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 15:37:46.392502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 15:37:46.724547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [29]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, witwidget


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/english-to-bengali-for-machine-translation/english to bengali.csv')

In [4]:
data = pd.read_table('/kaggle/input/bilingual-sentence-pairs/ben.txt', header=None)
data.columns = ['english_caption','bengali_caption','license']

In [ ]:
import pandas as pd

# Initialize an empty list to store data
data = []

# Specify the file path
file_path = '/combined_data.txt'

# Open the file and read line by line, limiting to the first 50000 lines
with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file):
        if line_number >= 20000:
            break  # Break loop after reading 50000 lines
        # Split each line based on the tab delimiter
        parts = line.strip().split('\t')
        
        # Take only the first column (assuming the desired data is in the first column)
        first_column = parts[0]
        
        # Append the first column to the data list
        data.append([first_column])

# Create a DataFrame from the collected data
ban = pd.DataFrame(data, columns=['bengali_caption'])



In [5]:
ban.head()

,bengali_caption
0,"এ জন্য সৎ, মেধাবী ও ত্যাগীদের বাছাই করে নেতৃত্..."
1,"আমার মা,থমকে গেল এক মুহূর্তের মধ্যে কি হয়ে গেল।"
2,"বিপুল প্রতিদন্দ্বিতার মধ্যে ৩ কেন্দ্রে ৪,৫ ও ৬..."
3,ওজিওয়াটস অ্যাপ্লিকেশন আপনাকে দুটি পৃথক ফোন নম...
4,অনুষ্ঠানে সভাপতিত্ব করবেন আবৃত্তিশিল্পী ও স্বা...


In [6]:
import pandas as pd

# Initialize an empty list to store data
data = []

# Specify the file path
file_path = '/kaggle/input/english-to-bengali-dataset/1_Eng.txt'

# Open the file and read line by line, limiting to the first 50000 lines
with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file):
        if line_number >= 20000:
            break  # Break loop after reading 50000 lines
        # Split each line based on the tab delimiter
        parts = line.strip().split('\t')
        
        # Take only the first column (assuming the desired data is in the first column)
        first_column = parts[0]
        
        # Append the first column to the data list
        data.append([first_column])

# Create a DataFrame from the collected data
eng = pd.DataFrame(data, columns=['english_caption'])



In [4]:
eng.head()

NameError: name 'eng' is not defined

In [8]:
df.head()

,english_caption,bengali_caption
0,a child in a pink dress is climbing up a set o...,একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্...
1,a girl going into a wooden building .,একটি মেয়ে শিশু একটি কাঠের বাড়িতে ঢুকছে
2,a little girl climbing into a wooden playhouse .,একটি বাচ্চা তার কাঠের খেলাঘরে উঠছে ।
3,a little girl climbing the stairs to her playh...,ছোট মেয়েটি তার খেলার ঘরের সিড়ি বেয়ে উঠছে
4,a little girl in a pink dress going into a woo...,গোলাপি জামা পড়া ছোট একটি মেয়ে একটি কাঠের তৈরি...


In [8]:
combined_df = pd.concat([eng, ban], axis=1)

In [9]:
combined_df.head()

,english_caption,bengali_caption
0,"For this, some of the party's middle-rung lead...","এ জন্য সৎ, মেধাবী ও ত্যাগীদের বাছাই করে নেতৃত্..."
1,My mother stopped what had happened in a moment.,"আমার মা,থমকে গেল এক মুহূর্তের মধ্যে কি হয়ে গেল।"
2,"Mim Chowdhury (Helicopter), daughter of the la...","বিপুল প্রতিদন্দ্বিতার মধ্যে ৩ কেন্দ্রে ৪,৫ ও ৬..."
3,The OGWats app allows you to use two separate ...,ওজিওয়াটস অ্যাপ্লিকেশন আপনাকে দুটি পৃথক ফোন নম...
4,"Ashraful Alam, a recitation artist and the voi...",অনুষ্ঠানে সভাপতিত্ব করবেন আবৃত্তিশিল্পী ও স্বা...


In [10]:
print(combined_df['english_caption'][1])
print('\n')
print(combined_df['bengali_caption'][1])

My mother stopped what had happened in a moment.


আমার মা,থমকে গেল এক মুহূর্তের মধ্যে কি হয়ে গেল।


In [13]:
data.head()

,english_caption,bengali_caption,license
0,Go.,যাও।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,যান।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,যা।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Run!,পালাও!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,পালান!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [5]:
data.drop(['license'], inplace = True, axis = 1)

In [6]:
df1 = pd.concat([data, df], ignore_index = True)

In [6]:
df1.head()

,english_caption,bengali_caption
0,Go.,যাও।
1,Go.,যান।
2,Go.,যা।
3,Run!,পালাও!
4,Run!,পালান!


In [6]:
final_df = pd.concat([df1, combined_df], ignore_index = True)

NameError: name 'df1' is not defined

In [18]:
final_df.head()

,english_caption,bengali_caption
0,Go.,যাও।
1,Go.,যান।
2,Go.,যা।
3,Run!,পালাও!
4,Run!,পালান!


In [19]:
print(final_df.isnull().sum())
print(final_df.duplicated().sum())

english_caption    0
bengali_caption    0
dtype: int64
0


# Function To Preprocess English Sentence

In [7]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [8]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English Sentence'''
    sent = sent.lower()
    sent = sent.replace("'", '')  # Changed .sub() to .replace()
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)
    
    sent = sent.strip()
    sent = re.sub(" +", " ", sent)
    return sent

In [9]:
def preprocess_ban_sentence(sent):
    '''Function to preprocess Bengali Sentence'''  # Corrected function description
    sent = sent.replace("'", '')  # Changed .sub() to .replace()
    sent = sent.replace("।", '')
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)
    
    sent = sent.strip()
    sent = re.sub(" +", " ", sent)
    sent = "startseq " + sent + " endseq"  # Added space before <end>
    
    return sent

Generate Pairs of Clean English and Bangla Sentences

In [10]:
df1['english_caption'] = df1['english_caption'].apply(preprocess_eng_sentence)
df1['bengali_caption'] = df1['bengali_caption'].apply(preprocess_ban_sentence)

In [11]:
df1.head()

,english_caption,bengali_caption
0,go,startseq যাও endseq
1,go,startseq যান endseq
2,go,startseq যা endseq
3,run,startseq পালাও endseq
4,run,startseq পালান endseq


In [12]:
max_length = 30

# Text to Vector Transformation

* First we will build the vocabulary for both the source and target sentences
* We will use this vocabulary to index each word in the sentence and convert it into it's numeric vector representation
* Vocabulary should only be build on the training sentences and not the test sentences, as test sentences might contain some new words. These new words are tyically represented as out vocabulary tokens.

In [13]:
eng_sentences = df1['english_caption'].tolist()
ban_sentences = df1['bengali_caption'].tolist()

In [14]:
n = len(eng_sentences)
split_index1 = int(0.95 * n)
split_index2 = int(0.98 * n)

train_eng_sents = eng_sentences[:split_index2]
test_eng_sents = eng_sentences[split_index2:]

train_ban_sents = ban_sentences[:split_index2]
test_ban_sents = ban_sentences[split_index2:]

In [15]:
len(train_eng_sents), len(test_eng_sents)

(42529, 868)

# Building Vocabulary

In [16]:
eng_tokenizer = Tokenizer(oov_token = '<OOV>')
eng_tokenizer.fit_on_texts(train_eng_sents)
eng_vocab_size = len(eng_tokenizer.word_index) +1

ban_tokenizer = Tokenizer()
ban_tokenizer.fit_on_texts(train_ban_sents)
ban_vocab_size = len(ban_tokenizer.word_index) + 1

print(f"English Vocab Size: {eng_vocab_size}\nBangla Vocab Size: {ban_vocab_size}")


English Vocab Size: 9372
Bangla Vocab Size: 17443


# Text Vectorization

* Since all the sentences wil not be of the same length, we will pad them to be of the maximum length that we decided earlier

* Once the text vectorization step is done we will split them into their input and output values

In [17]:
eng_sequences = eng_tokenizer.texts_to_sequences(train_eng_sents)
ban_sequences = ban_tokenizer.texts_to_sequences(train_ban_sents)

source_seqs = pad_sequences(eng_sequences, maxlen = max_length, padding = 'post')
target_seqs = pad_sequences(ban_sequences, maxlen = max_length, padding = 'post')

In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices((source_seqs[:split_index1], target_seqs[:split_index1]))
train_dataset = train_dataset.shuffle(buffer_size=split_index1).batch(32, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((source_seqs[split_index1:], target_seqs[split_index1:]))
val_dataset = val_dataset.batch(32, drop_remainder = True)

# Custome Loss Function

* we will use a custome lss funtion where we will mask out whenever os are present in the output labels

In [19]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_obj(real, pred)
    
    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

# Attention Mechanism

Attention mechanisms are a type of neural network layer that can be added to deep learing models. They allow the model to focus on specific parts of input  by assigning different weights to different weights to different parts of the input. This weighting is typically based on the relevence of each part of the input to the task at hand. Similary for the task of Neural Machine  Translation, the target sentence word will be alligned with each of the source sentence words to identify it's relevance with all the words

**Additive Attention -** This mechanism refers to Dzmitry Bahdanau's work title "Neural Machine Translation by Jointly Learning to Align and Translate". This, this tecnique is also known as Bahdanau attention. The additive attention is implemented as follows

  * Addition of weeighted Decoder hidden state and Encoder hidden states
  * Tanh function is applied over this addition
  * Output of step 2 is passed through another linear layer
  * Softmax function is applied to calculate the normalized allignment attention scores
  * Attention scores are applied to the encoder hidden state to indentify the importance of each source word with respect to the next output word
  
**Multiplicative Attention -** This method is proposed by Thang Luong in the work title 'Effective Approaches to Attention-based Neural Machine Translation'. It is build on top of additive attention (a.k.a Bahdanau attention). there are three scoring functions that we can choose from - dot, general , concat,(we have used dot in our code)
  * Matrix multiplication perfomed between Encoder hidden states and Deocder hidden state
  * Softmax function is applied t calculate the normalized allignment attention scores
  * Attention scores are applied to the Encoder hidden states to indentify the importance of each source word with respect to the next output word

In [20]:
class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, s_hidden, h_hidden):
        s_hidden = tf.expand_dims(s_hidden, axis=1)
        score = self.V(tf.nn.tanh(self.W1(s_hidden) + self.W2(h_hidden)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * h_hidden
        context_vector = tf.expand_dims(tf.reduce_sum(context_vector, axis=1),axis=1)

        return context_vector, attention_weights
    
class LuongAttention(Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
    
    def call(self, s_hidden, h_hidden):
        score = tf.matmul(h_hidden, s_hidden, transpose_b=True)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * h_hidden
        context_vector = tf.expand_dims(tf.reduce_sum(context_vector, axis=1),axis=1)
        
        return context_vector, attention_weights

# Sequence2Sequence Model With Attention Layer

## Encoder
  * It uses RNN layer and converts the input words to corresponding hidden vectors. Each vector represents the current word and the contex of the word. The encoder takes the input sequence, one token at a time, and uses an to update it's hidden state, which summarizes the information inthe input sequence. The finel hidden state (state_h in the code) of the encoder is then passed as the context vector to the decoder. Since we are using Bidirectional LSTM as our RNN, we will get 2 hidden ecoder states (forward_state_h and backward_state_h) and 2 cell states (forward_state_c and bacword_state_c). We concatenate them into one hidden and cell state (state_h, and state-c in the code)
  
  
## Decoder
  * It takes as input the hidden vector generated by the encoder, it's own hidden states, and the current word to preduce the next hidden vector and finally predict the next word. In out case, since it is a Bi LSTM, we also take the cell sate along with the hidden state. **When using the attention layer we first calculate the attention context vector by applying attention mechanism on the decoder hidden state and the encoder hidden states, once the context vector is preduced it is concatenated wirh the current word embeding and passed to the decoder to predict the next word. The decoder useds this new context vector and an initial hidden state to generate the output sequence, one token at a time. At each time step, the decoder uses the current hidden state, the attention alligned context vector, and the previous output token to generate a probability distribution over the possible next tokens. The token with the highest probability is then chosen as the output, and the process continues until the end of the output sequence (end) is reached.

In [21]:
class Encoder(Model):
    
    def __init__(self, vocab_size, embedding_dims, units):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dims, mask_zero=True)
        self.lstm = Bidirectional(LSTM(units, return_sequences=True, return_state=True))
        
    def call(self, x):
        x = self.embedding(x)
        enc_output, forward_h, forward_c, backward_h, backward_c = self.lstm(x)
        state_h = Concatenate()([forward_h, backward_h])
        state_c = Concatenate()([forward_c, backward_c])
        
        return enc_output, state_h, state_c
    
    def summary(self):
        
        x = Input(shape=(None,))
        model = Model(inputs=[x], outputs=self.call(x))
        return model.summary()

In [22]:
class Decoder(Model):
    
    def __init__(self, vocab_size, embedding_dims, units, use_additive=True):
        super(Decoder, self).__init__()
        self.units = units
        self.embedding = Embedding(vocab_size, embedding_dims, mask_zero=True)
        self.lstm = LSTM(units, return_sequences=True, return_state=True)
        self.attention = BahdanauAttention(units) if use_additive else LuongAttention(units)
        self.fc = TimeDistributed(Dense(vocab_size, activation='softmax'))
        
    def call(self, x, enc_output, state_h, state_c):
        x = self.embedding(x)
        context_vector, attn_weights = self.attention(state_h, enc_output)
        context_vector = Concatenate(axis=-1)([context_vector, x])
        dec_output, dec_h, dec_c = self.lstm(context_vector, initial_state=[state_h, state_c])
        output = self.fc(dec_output)
        
        return output, dec_h, dec_c, attn_weights
    
    def summary(self):
        x = Input(shape=(None,))
        enc_output = Input(shape=(None, self.units))
        state_h = Input(shape=(self.units,))
        state_c = Input(shape=(self.units,))
        model = Model(inputs=[x, enc_output, state_h, state_c], outputs=self.call(x, enc_output, state_h, state_c))
        return model.summary()

In [23]:
encoder = Encoder(eng_vocab_size, 512, 256)
decoder = Decoder(ban_vocab_size, 512, 512)

In [24]:
# decoder = Decoder(ban_vocab_size, 256, 256)

In [25]:
@tf.function
def train_step(x, y):
    loss_value = 0
    with tf.GradientTape() as tape:
        encoder_output, state_h, state_c  = encoder(x)
        dec_input = tf.expand_dims(y[:, 0], 1)
        
        for i in range(1, y.shape[1]):
            dec_output, state_h, state_c, attn_w = decoder(dec_input, encoder_output, state_h, state_c)
            loss_value += loss_function(y[:, i], dec_output[:, 0, :])
            dec_input = tf.expand_dims(y[:, i], 1)
    
    batch_loss = (loss_value / int(y.shape[1]))  
    weights = encoder.trainable_variables + decoder.trainable_variables
    grads = tape.gradient(loss_value, weights)
    optimizer.apply_gradients(zip(grads, weights))
    return batch_loss

@tf.function
def test_step(x, y):
    loss_value = 0
    encoder_output, state_h, state_c  = encoder(x)
    dec_input = tf.expand_dims(y[:, 0], 1)
        
    for i in range(1, y.shape[1]):
        dec_output, state_h, state_c, attn_w = decoder(dec_input, encoder_output, state_h, state_c)
        loss_value += loss_function(y[:, i], dec_output[:, 0, :])
        dec_input = tf.expand_dims(y[:, i], 1)
    
    batch_loss = (loss_value / int(y.shape[1]))
    return batch_loss

In [26]:
optimizer = tf.keras.optimizers.Adam()
epochs = 10
train_steps = np.sum([1 for i in train_dataset])
val_steps = np.sum([1 for i in val_dataset])  # Calculate number of validation steps
metrics_names = ['train_loss', 'val_loss']  # Include 'val_loss' in metrics_names
mon_val_loss = np.inf

for epoch in range(epochs):
    print("Epoch: ", epoch+1)
    training_loss = []
    validation_loss = []
    progBar = Progbar(train_steps, stateful_metrics=metrics_names)
    
    # Training loop
    for step, (X_batch_train, y_batch_train) in enumerate(train_dataset):
        train_loss = train_step(X_batch_train, y_batch_train)
        training_loss.append(train_loss)
        values = [('train_loss', train_loss)]
        progBar.update(step, values=values)
    
    avg_train_loss = np.average(training_loss)
    values = [('train_loss', avg_train_loss)]
    progBar.update(step, values)
    
    # Validation loop
    for step, (X_batch_val, y_batch_val) in enumerate(val_dataset):
        val_loss = test_step(X_batch_val, y_batch_val)
        validation_loss.append(val_loss)
    
    avg_val_loss = np.average(validation_loss)
    values.append(('val_loss', avg_val_loss))
    progBar.update(train_steps, values=values, finalize=True)
    
    # Check if validation loss improved
    if avg_val_loss < mon_val_loss:
        encoder.save_weights("encoder.weights.h5")
        decoder.save_weights("decoder.weights.h5")
        print(f"Model loss improved from {mon_val_loss:.5f} to {avg_val_loss:.5f} Checkpoint Created: encoder.weights.h5, decoder.weights.h5")
        mon_val_loss = avg_val_loss


Epoch:  1
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 268s 180ms/step - train_loss: 1.3215 - val_loss: 1.2156
Model loss improved from inf to 1.21557 Checkpoint Created: encoder.weights.h5, decoder.weights.h5
Epoch:  2
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 225s 174ms/step - train_loss: 0.9453 - val_loss: 1.0229
Model loss improved from 1.21557 to 1.02285 Checkpoint Created: encoder.weights.h5, decoder.weights.h5
Epoch:  3
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 225s 174ms/step - train_loss: 0.7527 - val_loss: 0.9488
Model loss improved from 1.02285 to 0.94878 Checkpoint Created: encoder.weights.h5, decoder.weights.h5
Epoch:  4
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 225s 174ms/step - train_loss: 0.6121 - val_loss: 0.9147
Model loss improved from 0.94878 to 0.91471 Checkpoint Created: encoder.weights.h5, decoder.weights.h5
Epoch:  5
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 224s 174ms/step - train_loss: 0.4970 - val_loss: 0.9082
Model loss improved from 0.91471 to 0.90821 Checkpoint Created: encoder.weights.h5, decoder.weights.h5
Epoc

In [27]:
encoder.save_weights("encoder1.weights.h5")
decoder.save_weights("decoder1.weights.h5")

In [34]:
encoder.load_weights('/kaggle/working/encoder1.weights.h5')
decoder.load_weights('/kaggle/working/decoder1.weights.h5')

In [35]:
def predict_sentence(en_input):
    eng_sequence = eng_tokenizer.texts_to_sequences([en_input])
    en_input = pad_sequences(eng_sequence, maxlen=max_length, padding='post')
    hidden_state, next_h, next_c = encoder(en_input)
    attn_plot = []

    curr_token = np.zeros((1,1))
    curr_token[0,0] = ban_tokenizer.word_index['startseq']
    pred_sentence = ''

    for i in range(max_length):
        output, next_h, next_c, attn_w = decoder(curr_token, hidden_state, next_h, next_c)
        attn_plot.append(attn_w.numpy().reshape(-1,))
        next_token = np.argmax(output[:, 0, :], axis=1)[0]
        next_word = ban_tokenizer.index_word[next_token]
        if next_word == 'endseq':
            break
        else:
            pred_sentence += ' ' + next_word
            curr_token[0,0] = next_token

    return pred_sentence.strip()

In [36]:
result = predict_sentence('i dont feel like eating anything now')
print('Predicted translation: {}'.format(result))    

Predicted translation: আমার এখন খেতে ইচ্ছে করছে না


In [37]:
import pickle
with open('eng_tokenizer.pkl','wb') as file:
    pickle.dump(eng_tokenizer, file)
    
import pickle
with open('ban_tokenizer.pkl','wb') as file:
    pickle.dump(ban_tokenizer, file)